<a href="https://colab.research.google.com/github/GreyKxtx/-Command-Injection-Payload-List/blob/master/AudioCompareAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.models import load_model
import librosa
import torch
from sklearn.metrics.pairwise import cosine_similarity


def pad_or_trim_feature(feature, target_shape):
    """
    Обрезка или дополнение признаков до нужной формы.

    :param feature: Исходный массив признаков.
    :param target_shape: Кортеж с желаемой формой (например, (40, 20)).
    :return: Массив признаков с новой формой.
    """
    current_shape = feature.shape

    # Если размер больше целевого, то обрезаем
    if current_shape[0] > target_shape[0]:
        feature = feature[:target_shape[0], :]
    if current_shape[1] > target_shape[1]:
        feature = feature[:, :target_shape[1]]

    # Если размер меньше, то добавляем нули
    if current_shape[0] < target_shape[0]:
        pad_width = target_shape[0] - current_shape[0]
        feature = np.pad(feature, ((0, pad_width), (0, 0)), mode='constant')

    if current_shape[1] < target_shape[1]:
        pad_height = target_shape[1] - current_shape[1]
        feature = np.pad(feature, ((0, 0), (0, pad_height)), mode='constant')

    return feature


def load_and_preprocess_audio(file_path):
    """Функция для загрузки и предобработки аудио"""
    # Загрузка аудиоданных с помощью librosa
    audio_data, sr = librosa.load(file_path, sr=None)

    # Извлечение признаков с использованием librosa
    mfcc = librosa.feature.mfcc(y=audio_data, sr=sr, n_mfcc=40)
    chroma = librosa.feature.chroma_stft(y=audio_data, sr=sr)
    spectral_contrast = librosa.feature.spectral_contrast(y=audio_data, sr=sr)
    tonnetz = librosa.feature.tonnetz(y=audio_data, sr=sr)

    # Нормализация признаков с использованием StandardScaler
    scaler = StandardScaler()
    mfcc_normalized = scaler.fit_transform(mfcc.T).T
    chroma_normalized = scaler.fit_transform(chroma.T).T
    spectral_contrast_normalized = scaler.fit_transform(spectral_contrast.T).T
    tonnetz_normalized = scaler.fit_transform(tonnetz.T).T

    return mfcc_normalized, chroma_normalized, spectral_contrast_normalized, tonnetz_normalized

def evaluate_similarity_with_multiple_tracks():
    # Загружаем модель
    model = load_model("/content/siamese_nn_2.keras")

    # Загрузка и предобработка первого аудиофайла (эталонного)
    mfcc_input_1, chroma_input_1, spectral_contrast_1, tonnetz_1 = load_and_preprocess_audio(
        '/content/data/source/metallica/Metallica - The Unforgiven (Official Music Video).wav')

    # Приведение первого трека к нужной форме
    mfcc_input_1 = pad_or_trim_feature(mfcc_input_1, (40, 20))
    chroma_input_1 = pad_or_trim_feature(chroma_input_1, (40, 12))

    mfcc_input_1 = np.expand_dims(mfcc_input_1, axis=0)
    chroma_input_1 = np.expand_dims(chroma_input_1, axis=0)

    # Список файлов для сравнения
    comparison_tracks = [
        '/content/data/source/electro/Boys Play Loud - Disco Btch (Gene K Remix) [vzvuke.net].mp3',
        '/content/data/source/electro/Daft Punk - Robot Rock (The White Screamed Shout Remix) [vzvuke.net].mp3',
        '/content/data/source/electro/Lady Gaga - Swine [vzvuke.net].mp3',
        '/content/data/source/electro/Madcap - So Strong [vzvuke.net].mp3',
        '/content/data/source/electro/Twofold - All Around [vzvuke.net].mp3',

        '/content/data/source/metallica/For Whom The Bell Tolls.wav',
        '/content/data/source/metallica/Metallica - The Unforgiven (Official Music Video).wav',
        '/content/data/source/metallica/Nothing Else Matters.wav',
        '/content/data/source/metallica/Sad But True.wav',

        '/content/data/source/pop/Adam & Eva - Где-То [vzvuke.net].mp3',
        '/content/data/source/pop/Katy Perry - Dark Horse [vzvuke.net].mp3',
        '/content/data/source/pop/В Двух Шагах - Сердце [vzvuke.net].mp3',
        '/content/data/source/pop/За деньги - Уехала в Сочи [vzvuke.net].mp3',
        '/content/data/source/pop/Юлия Беретта - Сколько Можно.mp3',
    ]



    for track in comparison_tracks:

        mfcc_input_2, chroma_input_2, spectral_contrast_2, tonnetz_2 = load_and_preprocess_audio(track)

        mfcc_input_2 = pad_or_trim_feature(mfcc_input_2, (40, 20))
        chroma_input_2 = pad_or_trim_feature(chroma_input_2, (40, 12))

        mfcc_input_2 = np.expand_dims(mfcc_input_2, axis=0)
        chroma_input_2 = np.expand_dims(chroma_input_2, axis=0)

        prediction_1 = model.predict([mfcc_input_1, chroma_input_1])
        prediction_2 = model.predict([mfcc_input_2, chroma_input_2])

        cosine_sim = cosine_similarity(prediction_1, prediction_2)[0][0]

        euclidean_dist = np.linalg.norm(prediction_1 - prediction_2)

        # Проверяем евклидово расстояние и выводим результат на основе порога
        if euclidean_dist <= 0.03:
            status = "Похожие"
        else:
            status = "Непохожие"

        # Выводим результаты
        print(f"Сравнение с {os.path.basename(track)}:")
        print("Евклидово расстояние: {:.2f}".format(euclidean_dist))
        print(f"Статус: {status}")
        print("-" * 50)

def main():
    evaluate_similarity_with_multiple_tracks()

if __name__ == '__main__':
    main()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Сравнение с Boys Play Loud - Disco Btch (Gene K Remix) [vzvuke.net].mp3:
Евклидово расстояние: 0.01
Статус: Похожие
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Сравнение с Daft Punk - Robot Rock (The White Screamed Shout Remix) [vzvuke.net].mp3:
Евклидово расстояние: 0.03
Статус: Похожие
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Сравнение с Lady Gaga - Swine [vzvuke.net].mp3:
Евклидово расстояние: 0.02
Статус: Похожие
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Сравнение с Madcap - So Strong [vzvuke.net].mp3:
Евклидово расстояние: 0.01
Статус: Похожие
--------------------------------------------------
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [9]:
!pip install librosa
!pip install keras
!pip install scikit-learn


In [ ]:
from google.colab import drive
drive.mount('/content/drive')